In [2]:
!pip install earthengine-api

In [3]:
import ee

In [5]:
# Authenticate to Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-user-001')   #Replace 'your-project-id' with the ID of your Earth Engine project.

In [6]:
# Define the geometry for the study area
area = ee.Geometry.Polygon(
        [[[95.6648788623047, 23.806941286545687],
          [95.6648788623047, 23.774269771886978],
          [95.70058442871095, 23.774269771886978],
          [95.70058442871095, 23.806941286545687]]])

In [7]:
# Set time frame
prefire_start = '2024-02-04'
prefire_end = '2024-02-05'
postfire_start = '2024-03-25'
postfire_end = '2024-03-26'

In [8]:
# Select satellite platform (Sentinel-2 or Landsat 8)
platform = 'S2'  # 'S2' for Sentinel-2, 'L8' for Landsat 8

In [22]:
# Define satellite platform specific image collection
if platform.upper() == 'S2':
    ImCol = 'COPERNICUS/S2_SR'
    mask_function = None  # No cloud masking for Sentinel-2
    pl = 'Sentinel-2'
else:
    ImCol = 'LANDSAT/LC08/C01/T1_SR'
    mask_function = None  # No cloud masking for Landsat 8
    pl = 'Landsat 8'

# Print selected platform and dates
print(f"Data selected for analysis: {pl}")
print(f"Fire incident occurred between {prefire_end} and {postfire_start}")

Data selected for analysis: Sentinel-2
Fire incident occurred between 2024-02-05 and 2024-03-25


In [23]:
# Load satellite imagery
imagery = ee.ImageCollection(ImCol)

In [24]:
# Filter imagery by date and location
prefireImCol = imagery.filterDate(prefire_start, prefire_end).filterBounds(area)
postfireImCol = imagery.filterDate(postfire_start, postfire_end).filterBounds(area)

In [26]:
# Apply cloud mask (no cloud mask in this work)
prefire_CM_ImCol = prefireImCol
postfire_CM_ImCol = postfireImCol

In [27]:
# Mosaic and clip images
pre_mos = prefireImCol.mosaic().clip(area)
post_mos = postfireImCol.mosaic().clip(area)
pre_cm_mos = prefire_CM_ImCol.mosaic().clip(area)
post_cm_mos = postfire_CM_ImCol.mosaic().clip(area)

In [28]:
# Calculate NBR for pre- and post-fire images
preNBR = pre_cm_mos.normalizedDifference(['B8', 'B12'])
postNBR = post_cm_mos.normalizedDifference(['B8', 'B12'])

In [29]:
# Calculate dNBR (difference between pre- and post-fire NBR)
dNBR_unscaled = preNBR.subtract(postNBR)
dNBR = dNBR_unscaled.multiply(1000)

In [30]:
# Define thresholds for burn severity classification
thresholds = ee.Image([-1000, -251, -101, 99, 269, 439, 659, 2000])
classified = dNBR.lt(thresholds).reduce('sum').toInt()

In [31]:
# Calculate burned area statistics
pixstats = classified.reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=area,
    scale=30)
allpixels = ee.Number(pixstats.get('sum'))

In [32]:
# Prepare legend
palette = ['7a8737', 'acbe4d', '0ae042', 'fff70b', 'ffaf38', 'ff641b', 'a41fd6', 'ffffff']
names = ['Enhanced Regrowth, High', 'Enhanced Regrowth, Low', 'Unburned', 'Low Severity',
         'Moderate-low Severity', 'Moderate-high Severity', 'High Severity', 'NA']

In [40]:
import geemap

# Create a Map object
Map = geemap.Map()

# Display the map with layers
Map.addLayer(pre_mos, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Pre-fire image')
Map.addLayer(post_mos, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Post-fire image')
Map.addLayer(dNBR, {'min': -1000, 'max': 1000, 'palette': ['white', 'black']}, 'dNBR greyscale')

# Define an SLD style of discrete intervals to apply to the image in Python format
sld_intervals = (
    '<RasterSymbolizer>'
    '<ColorMap type="intervals" extended="false" >'
    '<ColorMapEntry color="#ffffff" quantity="-500" label="-500"/>'
    '<ColorMapEntry color="#7a8737" quantity="-250" label="-250" />'
    '<ColorMapEntry color="#acbe4d" quantity="-100" label="-100" />'
    '<ColorMapEntry color="#0ae042" quantity="100" label="100" />'
    '<ColorMapEntry color="#fff70b" quantity="270" label="270" />'
    '<ColorMapEntry color="#ffaf38" quantity="440" label="440" />'
    '<ColorMapEntry color="#ff641b" quantity="660" label="660" />'
    '<ColorMapEntry color="#a41fd6" quantity="2000" label="2000" />'
    '</ColorMap>'
    '</RasterSymbolizer>'
)
# Add the image to the map using both the color ramp and interval schemes.
Map.addLayer(dNBR.sldStyle(sld_intervals), {}, 'dNBR classified');

# Zoom in to a specific location
Map.setCenter(95.68071482296116, 23.791234365125657, 15)  # Replace latitude, longitude, and zoom level with your desired values,23.791234365125657, 95.68071482296116

# Display the map
Map

Map(center=[23.791234365125657, 95.68071482296116], controls=(WidgetControl(options=['position', 'transparent_…

In [41]:
# Export dNBR image to Google Drive
id = dNBR.id().getInfo()
task = ee.batch.Export.image.toDrive(image=dNBR, scale=30, description='dNBR', fileNamePrefix='dNBR', region=area)
task.start()

In [42]:
# Print statistics
print('Burned Area by Severity Class:')
for i, name in enumerate(names):
    pix = ee.Number(classified.eq(i).reduceRegion(reducer=ee.Reducer.count(), geometry=area, scale=30).get('sum'))
    hect = pix.multiply(900).divide(10000)
    perc = pix.divide(allpixels).multiply(10000).round().divide(100)
    print(f'{name}: Pixels={pix.getInfo()}, Hectares={hect.getInfo()}, Percentage={perc.getInfo()}')

Burned Area by Severity Class:
Enhanced Regrowth, High: Pixels=15972, Hectares=1437.48, Percentage=100
Enhanced Regrowth, Low: Pixels=15972, Hectares=1437.48, Percentage=100
Unburned: Pixels=15972, Hectares=1437.48, Percentage=100
Low Severity: Pixels=15972, Hectares=1437.48, Percentage=100
Moderate-low Severity: Pixels=15972, Hectares=1437.48, Percentage=100
Moderate-high Severity: Pixels=15972, Hectares=1437.48, Percentage=100
High Severity: Pixels=15972, Hectares=1437.48, Percentage=100
NA: Pixels=15972, Hectares=1437.48, Percentage=100


In [43]:
# Export the map as an HTML file
html_file = "burnseveritymap_kawlin.html"
Map.to_html(html_file)

# Display the path to the HTML file
print(f"Map exported to: {html_file}")

Map exported to: burnseveritymap_kawlin.html
